In [1]:
import pandas as pd
from zipfile import ZipFile
from  pathlib import Path
import calendar
import datetime
import pickle

In [48]:
path = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\codes\periods.xlsx')

In [2]:
new_data =pd.read_parquet('data/new_data_merged/new_data.pq')
col_check = {}
for col in new_data.columns:
    if new_data[col].isna().sum():
        col_check[col] = new_data[col].isna().sum()
print(col_check)
      

{}


In [ ]:
new_data

In [4]:
bad_file = new_data[new_data['cat_segment'].isna()]['file'].unique()  
bad_file

array(['out_datatile/diapers_socdem-new24.parquet'], dtype=object)

In [98]:
bad_file

array(['out_datatile/diapers_socdem-new24.parquet'], dtype=object)

In [6]:
features_group = pd.read_excel(r'.\codes\feature_groups.xlsx')
demo_order = pd.read_excel(r'.\codes\demo_order.xlsx')
demo_order['buyers_gr_label'] = demo_order['buyers_gr_label'].astype(str)

In [7]:
def get_category(name_of_file):
    cat_dict = {
    'br':'MALE B&R', 'deterg': 'Laundry Detergents',
    'diapers': 'Diapers','femcare': 'Feminine Care', 
    'haircond': 'Hair Conditioners', 'shampoo':'Shampoos'}
    for k in cat_dict.keys():
        if name_of_file.count(k):
            return cat_dict[k]
    else:
        raise ValueError('В названии файла нет категории', name_of_file)    

In [54]:
with open('data/tmp/old_data.pickle', 'rb') as f:
    old_data = pickle.load(f)

In [8]:
dfs = []
for n in range(len(bad_file)):
  paths = f"data/tmp/7z/{bad_file[n]}"
  aaa = pd.read_parquet(paths)
  df_in = aaa.rename(
  {'Category Name':'Product Name', 'Buyer Group Name':'buyers_gr_label'}, 
  axis=1)
  df_in['Category Name'] = get_category(paths)
  if ('Segment' not in df_in.columns):
    print("нет сегмента")


  i =df_in.shape[0]
  df_in = df_in.merge(
  features_group,#.drop(columns='Product Name'), 
  left_on=['Category Name', 'Product Name'], 
  right_on=['Category Name','full_label' ], #'full_label'
  how='left')#, validate='many_to_one')
  df_in['file'] = paths
  print('Добавилось строк', df_in.shape[0] - i)
  dfs.append(df_in)

suka = pd.concat(dfs, ignore_index=True)

Добавилось строк 0


In [15]:
paths = f"data/tmp/7z/{bad_file[0]}"
aaa = pd.read_parquet(paths)

In [18]:
aaa['Segment'].unique()

array(['Total Diapers', 'POME (0,1,2)', 'Non POME', 'Taped',
       'Total Diapers size', 'Non-Taped'], dtype=object)

In [103]:
what =set(suka['buyers_gr_label'].str.lower().unique())- set(demo_order['buyers_gr_label'].str.lower())

In [104]:
what

set()

In [74]:
suka['buyers_gr_label'] = suka['buyers_gr_label'].str.lower()

In [77]:
suka[suka['buyers_gr_label'].isin(['1','2','3'])]['buyers_gr_label'] = suka[suka['buyers_gr_label'].isin(['1','2','3'])]['buyers_gr_label'].astype(int)

C:\Users\nyzhur\AppData\Local\Temp\ipykernel_18652\3273523086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suka[suka['buyers_gr_label'].isin(['1','2','3'])]['buyers_gr_label'] = suka[suka['buyers_gr_label'].isin(['1','2','3'])]['buyers_gr_label'].astype(int)


In [85]:
eb = suka[suka['buyers_gr_label'].isin(['1','2','3'])]['buyers_gr_label'].astype(int)

In [93]:
demo_order['buyers_gr_label'] = demo_order['buyers_gr_label'].astype(str)

In [94]:
type(demo_order['buyers_gr_label'].iloc[9])

str

In [78]:
set(suka['buyers_gr_label'].unique())- set(demo_order['buyers_gr_label'].str.lower())

{'1', '2', '3'}

In [73]:
for i in what:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>


In [67]:
suka['buyers_gr_label'].str.lower().unique()

array(['total demographics', '500 thd - 1 mln population',
       'family with children 6-12 year', '2',
       'no children <20 years (rest)', 'high affluency', '3',
       '30-39 years', 'moscow', '10 thd - 500 thd population',
       'below average affluency', '1mln+ population',
       'children 6-14 years', 'family with children <18',
       'employed full-time', 'family with children 0-3 years',
       '18-29 years', 'rural area', 'family with children 13-18 yea',
       'lifestages', 'senior singles/couples without',
       'size of households', '1', 'retired singles/couples withou',
       'not employed', 'st.petersburg', 'family with children <6 years',
       'children <6 years', 'above average affluency',
       'mature singles/couples without', '40-49 years', 'occupation',
       'young singles/couples without', '4+', '50-59 years',
       'low affluency', 'age', 'affluency', 'children',
       'children 15-19 years', '60+ years', 'geographical area',
       'employed part-

In [ ]:
bad_col = {}
for col in suka.columns:
    if suka[col].isna().sum():
       bad_col[col] = suka[col].isna().sum()
bad_col                

{'Segment': 1168619}

In [29]:
blyad = suka[suka['Segment'].isna()]['file'].unique()
blyad

array(['data/tmp/7z/out_datatile/femcare_socdem-new24.parquet',
       'data/tmp/7z/out_datatile/shampoo_socdem-new24.parquet'],
      dtype=object)

In [46]:
suka.columns

Index(['Product Name_x', 'Segment', 'position_name_shop', 'value', 'volume_gl',
       'volume_SU', 'volume_pack', 'trips_raw', 'trips', 'buyers',
       'buyers_raw', 'population', 'penetration', 'frequency',
       'spend_per_buyer', 'spend_per_trip', 'volume_per_buyer_g',
       'volume_per_buyer_su', 'volume_per_buyer_pack', 'volume_per_trip_g',
       'volume_per_trip_su', 'volume_per_trip_pack', 'avg_price_g',
       'avg_price_su', 'avg_price_pack', 'avg_pack_size', 'month_end',
       'duration', 'buyers_gr_label', 'year', 'month', 'Category Name',
       'Vendor Product ID', 'Product Name_y', 'feature', 'feature_code',
       'category_code', 'product_code', 'full_label', 'product_hier',
       'product_lvls', 'file'],
      dtype='object')

In [44]:
suka['buyers_gr_label'].unique()

array(['TOTAL DEMOGRAPHICS'], dtype=object)

In [43]:

with open(r'data\tmp\bad_codes.pickle', 'rb') as f:
  values_no_code = pickle.load(f)     

EOFError: Ran out of input